# Data Cleaning and Feature Engineering

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import re
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 100) # to look at more rows of data later
pd.set_option('display.max_columns', 100) # to expand columns view so that all can be seen later

In [2]:
# Load datasets
train_df = pd.read_csv('../datasets/train.csv')
test_df = pd.read_csv('../datasets/test.csv')

## Pre-Cleaning Phase:

We will lowercase and remove all spacings and special characters.<br>
After which we will verify and check that the columns in both train and test dataset are the same (with the exception of saleprice in train data)

In [3]:
# lowercase and remove spacings + special characters (in this case "/") in column names
train_df.columns = train_df.columns.str.lower().str.replace(' ','').str.replace('/','')
test_df.columns = test_df.columns.str.lower().str.replace(' ','').str.replace('/','')

In [4]:
missingcol = [i for i in train_df.columns if (i not in test_df) and i != 'saleprice']
print(missingcol)

[]


## Data Cleaning

### Check uniqueness of data

Check if id and pid are unique in each dataset, if yes then we will drop the PID column and use ID as the index.

In [5]:
def check_drop_ids(df):
    if df['id'].nunique() == df.shape[0]:
        df.set_index('id',inplace=True)
    if df['pid'].nunique() == df.shape[0]:
        df.drop(['pid'],axis=1,inplace=True)

check_drop_ids(train_df)
check_drop_ids(test_df)

### Logic check

We will check if the data fulfills the logical requirements, and we will drop them if required.

In [6]:
# Logic check for Yr Sold >= Garage Yr Blt/Year Built for train_df

train_df[(train_df['yrsold'] < train_df['garageyrblt']) | (train_df['yrsold'] < train_df['yearbuilt'])]

,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremodadd,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,heatingqc,centralair,electrical,1stflrsf,2ndflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,saleprice
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2261,20,RL,68.0,8298,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,1Story,8,5,2006,2007,Hip,CompShg,VinylSd,VinylSd,NaN,NaN,Gd,TA,PConc,Gd,TA,Av,GLQ,583.0,Unf,0.0,963.0,1546.0,GasA,Ex,Y,SBrkr,1564,0,0,1564,0.0,0.0,2,0,2,1,Ex,6,Typ,1,Gd,Attchd,2207.0,RFn,2.0,502.0,TA,TA,Y,132,0,0,0,0,0,NaN,NaN,NaN,0,9,2007,New,267300
2181,20,RL,128.0,39290,Pave,NaN,IR1,Bnk,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,10,5,2008,2009,Hip,CompShg,CemntBd,CmentBd,Stone,1224.0,Ex,TA,PConc,Ex,TA,Gd,GLQ,4010.0,Unf,0.0,1085.0,5095.0,GasA,Ex,Y,SBrkr,5095,0,0,5095,1.0,1.0,2,1,2,1,Ex,15,Typ,2,Gd,Attchd,2008.0,Fin,3.0,1154.0,TA,TA,Y,546,484,0,0,0,0,NaN,NaN,Elev,17000,10,2007,New,183850


The above checks show that the garageyrblt for that data row for id 2261 in train_df is invalid as it is 2207, far beyond our current year.
This is likely a typo, so we will change the value to the most probable year, which in this case is 2007. 

In [7]:
train_df.loc[train_df['garageyrblt'] == 2207,'garageyrblt'] = 2007

For index 2181 of train_df, it seems that the house is sold even before the house is built.
Assumption here is that the house needs to be built before it can be sold, so we will drop this row.


In [8]:
train_df.drop(index=2181,inplace=True)

In [9]:
# Logic check for Yr Sold >= Garage Yr Blt/Year Built for test_df
test_df[(test_df['yrsold'] < test_df['garageyrblt']) | (test_df['yrsold'] < test_df['yearbuilt'])]

,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremodadd,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,heatingqc,centralair,electrical,1stflrsf,2ndflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [10]:
# Logic check for Year Remod/Add >= Year Built for train_df
train_df[(train_df['yearremodadd'] < train_df['yearbuilt'])]

,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremodadd,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,heatingqc,centralair,electrical,1stflrsf,2ndflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,saleprice
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
851,20,RL,65.0,10739,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,1Story,7,5,2002,2001,Gable,CompShg,VinylSd,VinylSd,BrkFace,68.0,Gd,TA,PConc,Gd,TA,No,GLQ,1259.0,Unf,0.0,172.0,1431.0,GasA,Ex,Y,SBrkr,1444,0,0,1444,1.0,0.0,2,0,3,1,Gd,6,Typ,0,NaN,Attchd,2002.0,RFn,2.0,577.0,TA,TA,Y,144,40,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,203000


For id 851 in train_df, Year Remod/Add occurred before the Year Built, which is not the case logically. <br>
Decision: row to be dropped.

In [11]:
train_df.drop(index=851,inplace=True)

In [12]:
# Logic check for Year Remod/Add >= Year Built for test_df
test_df[(test_df['yearremodadd'] < test_df['yearbuilt'])]

,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremodadd,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,heatingqc,centralair,electrical,1stflrsf,2ndflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [13]:
# Logic Check for Total Bsmt SF = BsmtFin SF 1 + BsmtFin SF 2 + Bsmt Unf SF for train_df
train_df[train_df['bsmtfinsf1'] + train_df['bsmtfinsf2']+train_df['bsmtunfsf'] != train_df['totalbsmtsf']]

,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremodadd,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,heatingqc,centralair,electrical,1stflrsf,2ndflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,saleprice
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1342,20,RM,99.0,5940,Pave,NaN,IR1,Lvl,AllPub,FR3,Gtl,BrkSide,Feedr,Norm,1Fam,1Story,4,7,1946,1950,Gable,CompShg,MetalSd,CBlock,None,0.0,TA,TA,PConc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GasA,TA,Y,FuseA,896,0,0,896,NaN,NaN,1,0,2,1,TA,4,Typ,0,NaN,Detchd,1946.0,Unf,1.0,280.0,TA,TA,Y,0,0,0,0,0,0,NaN,MnPrv,NaN,0,4,2008,ConLD,79000


For index 1342, we see that the values for the features related to bsmt are null, which may indicate that this house may not have a basement.<br>
Decision: This will be handled later when we are filling up null values.

In [14]:
# Logic Check for Total Bsmt SF = BsmtFin SF 1 + BsmtFin SF 2 + Bsmt Unf SF for test_df
test_df[test_df['bsmtfinsf1'] + test_df['bsmtfinsf2']+ test_df['bsmtunfsf'] != test_df['totalbsmtsf']]

,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremodadd,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,heatingqc,centralair,electrical,1stflrsf,2ndflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [15]:
# Logic Check for Gr Liv Area = 1st Flr SF + 2nd Flr SF + Low Qual Fin SF for train_df
train_df[train_df['1stflrsf'] + train_df['2ndflrsf'] + train_df['lowqualfinsf'] != train_df['grlivarea']][['1stflrsf','2ndflrsf','lowqualfinsf','grlivarea']]

,1stflrsf,2ndflrsf,lowqualfinsf,grlivarea
id,,,,


In [16]:
# Logic Check for Gr Liv Area = 1st Flr SF + 2nd Flr SF + Low Qual Fin SF for test_df
test_df[test_df['1stflrsf'] + test_df['2ndflrsf'] + test_df['lowqualfinsf'] != test_df['grlivarea']]

,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremodadd,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,heatingqc,centralair,electrical,1stflrsf,2ndflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


### Filling of null values

For categorical null values, we will replace with the default NA/None.

Will look into whether it will make sense to replace with mode values instead.

In [17]:
# Writing a function to fill the null values for the categorical variables
# For now we will fill all null values with default NA/None value. Further exploration might be needed if this assumption is too generalistic.
def fill_cat_null(df,list_of_col):
    for i in list_of_col:
        if i == 'masvnrtype': # Based on data dictionary, the default none value for this case is None and not NA
            df[i] = df[i].fillna('None')
        else:
            df[i] = df[i].fillna('NA')
    return df

traincol_cat_null = [i for i in train_df.columns if (train_df[i].dtypes == np.object) & (train_df[i].count() < train_df.shape[0])]
testcol_cat_null = [i for i in test_df.columns if (test_df[i].dtypes == np.object) & (test_df[i].count() < test_df.shape[0])]

train_df = fill_cat_null(train_df,traincol_cat_null)
test_df = fill_cat_null(test_df,testcol_cat_null)

For numerical values, we will replace all null values to 0.

To look further later into whether will it be a better choice to consider taking the median/mean/mode to replace the null value

In [18]:
traincol_num_null = [i for i in train_df.columns if (train_df[i].dtypes == np.float64) & (train_df[i].count() < train_df.shape[0])]
testcol_num_null = [i for i in test_df.columns if (test_df[i].dtypes == np.float64) & (test_df[i].count() < test_df.shape[0])]
for i in traincol_num_null:
    train_df[i] = train_df[i].fillna(0)
    
for i in testcol_num_null:
    test_df[i] = test_df[i].fillna(0)


## Feature Engineering

### Conversion of column types for categorial data

As observed in our exploration of data, <b>MS Sub Class</b> and <b>Mo Sold</b> will need to be converted to string values as they are classified as categorial data and not numeric values.

In [19]:
def convert_to_string(df,list_of_column):
    for i in list_of_column:
        df[i] = df[i].astype(str)
    return df

train_df = convert_to_string(train_df,['mssubclass','mosold'])
test_df = convert_to_string(test_df,['mssubclass','mosold'])

### Calculate new values using existing data

From our exploration earlier, we can calculate values that we can include as our features during feature slection later. The following values will be calculated:

1. age_house = Yr Sold - Year Built<br>
This is to calculate the age of the house when it was sold
2. num_years_remod = Yr Sold - Year Remod/Add <br>
This is to calculate year difference between the year it was sold and the year it was remod.
3. age_garage = Yr Sold - Garage Yr Blt<br>
This is to calculate the age of the garage when it was sold. If age is 0, it means there is no garage present.
4. overallsf = Gr Liv Area + Total Bsmt SF <br>
This is to calculate the overall area of the house in square feet

In [20]:
def perform_all_calc(df):
    df['age_house'] =  df['yrsold'] - df['yearbuilt'] # calculate age of house
    df['years_after_remod'] = df['yrsold'] - df['yearremodadd'] #calculate years after remod
    # For the age of garage, as some of the garageyrblt was set to 0 due to it being null, 
    # age_garage will be 0 if garageyrblt is 0.
    df['age_garage'] = df['yrsold'] - df['garageyrblt']
    df.loc[df['garageyrblt'] == 0,'age_garage'] = 0
    df['overallsf'] = df['grlivarea'] + df['totalbsmtsf']
    return df

train_df = perform_all_calc(train_df)
test_df = perform_all_calc(test_df)


In [21]:
# Check for anomalies in calculated data
train_df[['age_house','years_after_remod','age_garage','overallsf']].describe()

,age_house,years_after_remod,age_garage,overallsf
count,2049.000000,2049.000000,2049.000000,2049.000000
mean,36.099073,23.605661,27.575891,2552.921425
std,30.207865,21.020909,25.154274,793.438169
min,0.000000,-1.000000,0.000000,334.000000
25%,7.000000,4.000000,4.000000,2010.000000
50%,34.000000,15.000000,22.000000,2457.000000
75%,54.000000,43.000000,46.000000,2998.000000
max,136.000000,60.000000,114.000000,11752.000000


In [22]:
test_df[['age_house','years_after_remod','age_garage','overallsf']].describe()

,age_house,years_after_remod,age_garage,overallsf
count,879.000000,879.000000,879.000000,879.000000
mean,37.291240,23.379977,29.424346,2537.282139
std,30.477921,20.486428,25.982356,792.419246
min,0.000000,-1.000000,0.000000,720.000000
25%,8.000000,5.000000,5.000000,1978.000000
50%,36.000000,15.000000,28.000000,2432.000000
75%,54.500000,41.000000,47.000000,2983.000000
max,129.000000,60.000000,110.000000,7814.000000


### Drop outliers

Those houses with Gr Liv Area > 4000 will be dropped.

In [23]:
train_df = train_df[train_df['grlivarea'] <= 4000]
# test_df = test_df[test_df['grlivarea'] <= 4000]

### Drop unnecessary columns

In [24]:
columns_to_drop = ['poolqc','miscfeature','alley','yrsold', 'yearbuilt', 'yearremodadd', 'garagecars',
                   'garageyrblt','bsmtfinsf1','bsmtfinsf2','bsmtunfsf','totalbsmtsf','1stflrsf','2ndflrsf',
                   'grlivarea','1stflrsf','2ndflrsf','lowqualfinsf']

train_df.drop(columns_to_drop, axis=1,inplace=True)
test_df.drop(columns_to_drop, axis=1,inplace=True)

### Encode ordinal data

The following data columns are identified as ordinal based on the data dictionary:

lotshape, utilities, landslope, overallqual,overallcond,exterqual,extercond, bsmtqual,bsmtcond, bsmtexposure, bsmtfintype1, bsmtfintype2, heatingqc, electrical, kitchenqual, functional, fireplacequ, garagefinish,garagequal,garagecond,paveddrive, fence

In [25]:
qualcond_val_dict = {'Ex': 5,
                   'Gd' : 4,
                   'TA' : 3,
                   'Fa' : 2,
                   'Po' : 1,
                   'NA' : 0 }

bsmttype_val_dict = {'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'NA':0}

encode_dict = {'lotshape': {'Reg': 4,'IR1':3,'IR2':2,'IR3':1},
              'utilities': {'AllPub':4,'NoSewr':3,'NoSeWa':2,'ELO':1},
              'landslope': {'Gtl':3,'Mod':2,'Sev':1,'NA':0},                    
              'exterqual': qualcond_val_dict,
              'extercond': qualcond_val_dict,
              'bsmtqual': qualcond_val_dict,
              'bsmtcond': qualcond_val_dict,
              'bsmtexposure': {'Gd':4,'Av':3,'Mn':2,'No':1,'NA':0},
              'bsmtfintype1': bsmttype_val_dict,
              'bsmtfintype2': bsmttype_val_dict,
              'heatingqc': qualcond_val_dict,
              'electrical': {'SBrkr':5,'FuseA':4,'FuseF':3,'FuseP':2,'Mix':1,'NA':0},
              'kitchenqual': qualcond_val_dict,
              'functional': {'Typ':8,'Min1':7,'Min2':6,'Mod':5,'Maj1':4,'Maj2':3,'Sev':2,'Sal':1,'NA':0},
              'fireplacequ': qualcond_val_dict,
              'garagefinish': {'Fin':3,'RFn':2,'Unf':1,'NA':0},
              'garagequal': qualcond_val_dict,
              'garagecond': qualcond_val_dict,
              'paveddrive': {'Y':3,'P':2,'N':1,'NA':0},
              'fence': {'GdPrv':4,'MnPrv':3,'GdWo':2,'MnWw':1,'NA':0}}

train_df.replace(encode_dict,inplace=True)
test_df.replace(encode_dict,inplace=True)

In [26]:
# Check type and change if necessary

def change_type(df,dict):
    for i in df.columns:
        if (df[i].dtypes == np.object) and (i in dict.keys()):
            df[i] = df[i].astype(int)
    return df

train_df = change_type(train_df,encode_dict)
test_df = change_type(test_df,encode_dict)


### Add dummies for categorical data

We will split the columns for the categorical data into dummy columns.
Once that is done, we will remove all the columns that are not present in either train/test dataset.

In [27]:
features_cat = [i for i in train_df.columns if (i != 'salesprice') and (train_df[i].dtypes == np.object)]
train_df = pd.get_dummies(train_df, columns=features_cat, drop_first=True)
test_df = pd.get_dummies(test_df, columns=features_cat, drop_first=True)

In [28]:
# Drop columns that does not exist in test_df/train_df except saleprice
for i in train_df.columns:
    if (i not in test_df.columns) and (i != 'saleprice'):
        train_df.drop([i],axis=1,inplace=True)

for i in test_df.columns:
    if i not in train_df.columns:
        test_df.drop([i],axis=1,inplace=True)

In [29]:
print(test_df.shape)
print(train_df.shape)

(879, 187)
(2048, 188)


Checkpoint: To save into a new csv and proceed to EDA and Feature Selection

In [30]:
train_df.to_csv('../datasets/train_cleaned.csv')
test_df.to_csv('../datasets/test_cleaned.csv')